# <center>Zaawansowana Eksploracja Danych</center>
## <center>Projekt z uczenia maszynowego</center>
### <center>Piotr Chojnacki 100743</center>

Celem drugiego projektu jest stworzenie dwóch klasyfikatorów danych krystalograficznych badanych w ramach analizy w języku R. Pierwszy klasyfikator powinien być nauczony na podstawie oryginalnego zbioru etykiet (res_name), a drugi na podstawie pogrupowanych etykiet (group_label), które dostarczy prowadzący.

#### Zakres pracy:

##### 0. Załączenie odpowiednich bibliotek:

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib

##### 1. Usuniętcie wierszy posiadających wartość zmiennej res_name równą: “DA”,“DC”,“DT”, “DU”, “DG”, “DI”,“UNK”, “UNX”, “UNL”, “PR”, “PD”, “Y1”, “EU”, “N”, “15P”, “UQ”, “PX4” lub “NAN”:

In [2]:
dataFrame = pd.read_csv('all_summary.txt', sep=";", na_values=["n/a"], low_memory=False, keep_default_na = False, index_col=0)

In [3]:
dataFrame.shape

(40309, 794)

In [4]:
bad_res_name = ["DA","DC","DT","DU","DG","DI","UNK","UNX","UNL","PR","PD","Y1","EU","N","15P","UQ","PX4","NAN"]
dataFrame = dataFrame[~dataFrame.res_name.isin(bad_res_name)]
dataFrame.shape

(40027, 794)

##### 2. Pozostawienie unikatowych par (pdb_code, res_name):

In [5]:
pairs = ["pdb_code", "res_name"]
dataFrame = dataFrame.drop_duplicates(subset=pairs, keep='last')
dataFrame.shape

(14132, 794)

##### 3. Stworzenie klasyfikatora uczonego na oryginalnym zbiorze etykiet (res_name) korzystajcego ze wszystkich klas, których liczność wynosi co najmniej 5 przykładów:

In [6]:
# Odfiltrowanie etykiet o liczności < 5:
dataFrame['unique'] = dataFrame.res_name.map(dataFrame.res_name.value_counts())
dataFrame = dataFrame[dataFrame.unique > 4]
dataFrame = dataFrame.drop('unique', 1)
dataFrame.shape

(11005, 794)

In [7]:
# Klasyfikator 1:

nums = ['int16', 'int32', 'int64']
numericDataFrame = dataFrame.select_dtypes(include=nums)
numericDataFrame.replace([np.inf, -np.inf], np.nan).dropna(axis=1).fillna(0)
res_name_DF = dataFrame['res_name']

param_grid = {'max_depth': np.arange(3, 10)}
tree = GridSearchCV(DecisionTreeClassifier(), param_grid)

tree.fit(numericDataFrame, res_name_DF)

joblib.dump(tree, '1.pkl') 

tree.grid_scores_

[mean: 0.36947, std: 0.00644, params: {'max_depth': 3},
 mean: 0.47015, std: 0.00757, params: {'max_depth': 4},
 mean: 0.58319, std: 0.00906, params: {'max_depth': 5},
 mean: 0.63371, std: 0.00918, params: {'max_depth': 6},
 mean: 0.67433, std: 0.00862, params: {'max_depth': 7},
 mean: 0.70232, std: 0.00567, params: {'max_depth': 8},
 mean: 0.72376, std: 0.00340, params: {'max_depth': 9}]

##### 4. Stworzenie klasyfikatora uczonego na pogrupowanych etykietach (group_label), z tych samych danych co pierwszy klasyfikator

In [8]:
# Klasyfikator 2:

nums = ['int16', 'int32', 'int64']
dataFrameGrouped = pd.read_csv("grouped_res_name.txt", sep=",", na_values=['n/a','NA','NaN','','nan'], low_memory=False, keep_default_na = False, index_col=0)
dataFrame['res_name_group'] = dataFrameGrouped['res_name_group']
res_name_DF = np.asarray(dataFrame["res_name_group"], dtype="|S6")

numericDataFrame = dataFrame.select_dtypes(include=nums)
numericDataFrame.replace([np.inf, -np.inf], np.nan).dropna(axis=1).fillna(0)

param_grid = {'max_depth': np.arange(3, 10)}
tree = GridSearchCV(DecisionTreeClassifier(), param_grid)
tree.fit(numericDataFrame, res_name_DF)

joblib.dump(tree, '2.pkl')

tree.grid_scores_

[mean: 1.00000, std: 0.00000, params: {'max_depth': 3},
 mean: 1.00000, std: 0.00000, params: {'max_depth': 4},
 mean: 1.00000, std: 0.00000, params: {'max_depth': 5},
 mean: 1.00000, std: 0.00000, params: {'max_depth': 6},
 mean: 1.00000, std: 0.00000, params: {'max_depth': 7},
 mean: 1.00000, std: 0.00000, params: {'max_depth': 8},
 mean: 1.00000, std: 0.00000, params: {'max_depth': 9}]